# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityWeatherDF = pd.read_csv("../WeatherPy/Data/CityWeatherData.csv")
cityWeatherDF.head()

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,gao,Mali,88,1579742459,14,16.64,1.64,76.96,10.58
1,1,mar del plata,Argentina,75,1579742199,94,-38.00,-57.56,69.80,5.82
2,2,yerbogachen,Russian Federation,51,1579742460,95,61.28,108.01,-32.91,4.56
3,3,dicabisagan,Philippines,75,1579742460,78,17.08,122.42,74.57,9.08
4,4,albany,Australia,1,1579742300,73,42.60,-73.97,27.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure 
gmaps.configure(api_key=gkey)

# get parts for heatmap
humidity = [float(h) for h in cityWeatherDF['Humidity']]
locations = cityWeatherDF[['Lat','Lng']]
locations = locations.rename(columns={'Lat':'Latitude','Lng':'Longitude'})

# add heatmap
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=max(humidity),
                                    point_radius=2)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# subset to only good weather
goodWeatherDF = cityWeatherDF[(cityWeatherDF['Max Temp'] > 65) & (cityWeatherDF['Max Temp'] < 80) & (cityWeatherDF['Humidity'] < 60) & (cityWeatherDF['Wind Speed'] < 8)]

goodWeatherDF = goodWeatherDF.reset_index()
goodWeatherDF

,index,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,94,94,podor,Senegal,92,1579742479,33,16.65,-14.96,73.40,2.24
1,95,95,diego de almagro,Chile,13,1579742480,45,-26.37,-70.05,67.35,4.94
2,148,148,kisangani,"Congo, The Democratic Republic of the",100,1579742492,55,0.52,25.21,71.28,1.92
3,171,171,opuwo,Namibia,48,1579742466,44,-18.06,13.84,70.05,2.89
4,179,179,lisala,"Congo, The Democratic Republic of the",82,1579742498,46,2.15,21.52,68.85,1.25
5,236,236,tokonou,Guinea,100,1579742283,51,9.65,-9.78,78.85,2.73
6,367,367,nicoya,Costa Rica,100,1579742541,57,10.15,-85.45,78.80,4.70
7,436,436,nueve de julio,Argentina,0,1579742558,52,-35.44,-60.88,76.01,1.01
8,447,447,odienne,Côte d'Ivoire,83,1579742561,29,9.51,-7.57,78.42,3.11
9,461,461,jalingo,Nigeria,34,1579742565,19,8.88,11.37,70.52,6.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# make hotel dataframe
hotel_df = goodWeatherDF
# make basic api url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": "1,1",
          "rankby": "distance",
          "type": "hotel",
          "key": gkey}

In [6]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    loc = f"{float(row['Lat'])},{float(row['Lng'])}"

    # add keyword to params dict
    params['location'] = loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['vicinity']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: podor.
Closest hotel is Station d'épuration de podor.
------------
Retrieving Results for Index 1: diego de almagro.
Closest hotel is electromecanico.
------------
Retrieving Results for Index 2: kisangani.
Closest hotel is Omakoy.
------------
Retrieving Results for Index 3: opuwo.
Closest hotel is Bakery Restaurant.
------------
Retrieving Results for Index 4: lisala.
Closest hotel is Hospital General De Lisala.
------------
Retrieving Results for Index 5: tokonou.
Closest hotel is Tokounou.
------------
Retrieving Results for Index 6: nicoya.
Closest hotel is Plantel Rassa Nicoya.
------------
Retrieving Results for Index 7: nueve de julio.
Closest hotel is Don Pedro Vinoteca.
------------
Retrieving Results for Index 8: odienne.
Closest hotel is DG Budget and finances- Odienne.
------------
Retrieving Results for Index 9: jalingo.
Closest hotel is Mosque.
------------
Retrieving Results for Index 10: khorixas.
Closest hotel is Damara Mopane Lodge.
--

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in goodWeatherDF.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))